In [1]:
import rpy2
print(rpy2.__version__)

2.9.0


In [2]:
from rpy2.robjects.packages import importr

In [3]:
sp = importr('sp')
gstat = importr('gstat')
intamap = importr('intamap')

In [4]:
from rpy2 import robjects

In [5]:
robjects.r('jet.colors  <- c("#00007F","blue","#007FFF","cyan","#7FFF7F","yellow","#FF7F00","red","#7F0000")')
robjects.r('col.palette <- colorRampPalette(jet.colors)')

R object with classes: ('function',) mapped to:
<SignatureTranslatedFunction - Python:0x7f3d47979c88 / R:0x5655501edbc8>

In [6]:
xy = robjects.r('xy <- read.table("radar_xy.csv",sep=",")')

In [7]:
nrow = robjects.r('nrow <- dim(xy)[1]')

In [8]:
RAD24 = robjects.r('RAD24 <- read.table("./radar_sent/radar_snap_24h_2011_08_05-00_00.csv",sep=",",colClasses="numeric")')

In [9]:
RAD24 = robjects.r('RAD24 <- as.numeric(as.vector(RAD24))')

In [10]:
RAD24 = robjects.r('RAD24 <- RAD24[6:length(RAD24)]')

In [11]:
RAD24

3.350000,5.650000,5.510000,6.240000,...,44.480000,41.030000,50.760000,45.730000


In [12]:
data = robjects.r('data <- data.frame(xy,RAD24)')

In [13]:
data

V1,V2,RAD24
3649.014148,1745.990418,3.350000
3650.003652,1745.945446,5.650000
3651.030377,1746.036880,5.510000


In [14]:
data.colnames = robjects.r('names(data) <- c("x","y","R")')

In [15]:
data

x,y,R
3649.014148,1745.990418,3.350000
3650.003652,1745.945446,5.650000
3651.030377,1746.036880,5.510000


In [16]:
robjects.r('coordinates(data) <- ~x+y')

R object with classes: ('formula',) mapped to:
<RObject - Python:0x7f3d474cabc8 / R:0x565551d7b740>

In [17]:
robjects.r('''
png("map.png",height=900,width=900)
ncuts <- 20
cuts <- seq(min(RAD24,na.rm=TRUE),max(RAD24,na.rm=TRUE),length=ncuts)
print(spplot(data["R"],xlab="East [m]",ylab="North [m]",key.space="right",cuts=cuts,region=TRUE,col.regions=col.palette(ncuts),main="Rainfall [mm]",scales=list(draw=TRUE)))
dev.off()
''')

1


In [18]:
wet = robjects.r('wet <- which(RAD24>0)')

In [19]:
robjects.r('isotropic_variogram <- variogram(R~1,data[wet,],width=2,cutoff=100)	# isotropic variogram with a class width of 2 km and a max. distance of 100 km (wet pixels only)')

np,dist,gamma,dir.hor,dir.ver,id
97340.000000,1.361011,21.909125,0.000000,0.000000,var1
342089.000000,2.980286,59.871361,0.000000,0.000000,var1
599568.000000,4.979613,105.459092,0.000000,0.000000,var1
781415.000000,6.968800,147.173973,0.000000,0.000000,var1
...,...,...,...,...,...
3368022.000000,92.964225,441.462909,0.000000,0.000000,var1


In [20]:
robjects.r('''
dist <- isotropic_variogram$dist	# range bins (inter-distances)
gam <- isotropic_variogram$gamma	# semivariance estimates for each range bin
np <- isotropic_variogram$np		# number of pairs for each range bin
''')

97340.000000,342089.000000,599568.000000,781415.000000,...,3368022.000000,3299515.000000,3249164.000000,3176925.000000


In [21]:
robjects.r('''
variogram_map <- variogram(R~1,data[wet,],width=2,cutoff=50,map=TRUE)
png("variogram_map.png",height=600,width=600)
print(plot(variogram_map))
dev.off()
''')

1


In [37]:
import pandas as pd
import numpy as np

In [40]:
coords = pd.read_csv('./radar_xy.csv', header=None)
coords.columns = ['x', 'y']
coords.head()

,x,y
0,3649.014148,1745.990418
1,3650.003652,1745.945446
2,3651.030377,1746.036880
3,3652.020031,1745.992300
4,3653.009769,1745.947910


In [43]:
rainfall = pd.read_csv('./radar_sent/radar_snap_24h_2011_08_05-00_00.csv', header=None)
rainfall = pd.DataFrame(rainfall.iloc[0,5::])
rainfall.index = np.arange(0,len(rainfall),1)
rainfall.columns = ['R']
rainfall['x'] = coords['x']
rainfall['y'] = coords['y']
rainfall.head()

,R,x,y
0,3.35,3649.014148,1745.990418
1,5.65,3650.003652,1745.945446
2,5.51,3651.030377,1746.036880
3,6.24,3652.020031,1745.992300
4,7.64,3653.009769,1745.947910


In [44]:
from rpy2.robjects import pandas2ri

In [58]:
mask = rainfall.R>0

In [61]:
rainfall = rainfall[mask]

In [49]:
pandas2ri.activate()

In [62]:
r_df = pandas2ri.py2ri(rainfall)

In [63]:
robjects.r.assign('mydata', r_df)

R,x,y
3.350000,3649.014148,1745.990418
5.650000,3650.003652,1745.945446
5.510000,3651.030377,1746.036880


In [64]:
robjects.r('head(mydata)')

,R,x,y
0,3.35,3649.014148,1745.990418
1,5.65,3650.003652,1745.945446
2,5.51,3651.030377,1746.036880
3,6.24,3652.020031,1745.992300
4,7.64,3653.009769,1745.947910
5,6.69,3653.955373,1746.004183


In [70]:
robjects.r('''
mydata <- data.frame(mydata)
coordinates(mydata) <- ~x+y
''')

R object with classes: ('formula',) mapped to:
<RObject - Python:0x7f3d20406548 / R:0x56555106fc60>

In [71]:
robjects.r('myiso <- variogram(R~1,mydata,width=2,cutoff=100)')

,np,dist,gamma,dir.hor,dir.ver,id
1,97340.0,1.361011,21.909125,0.0,0.0,var1
2,342089.0,2.980286,59.871361,0.0,0.0,var1
3,599568.0,4.979613,105.459092,0.0,0.0,var1
4,781415.0,6.968800,147.173973,0.0,0.0,var1
5,1041462.0,8.972910,183.165594,0.0,0.0,var1
6,1148166.0,10.929269,216.566412,0.0,0.0,var1
7,1477391.0,12.936641,251.996652,0.0,0.0,var1
8,1567984.0,14.958394,285.561328,0.0,0.0,var1
9,1800696.0,16.961547,318.288638,0.0,0.0,var1
10,1956390.0,18.977989,351.517847,0.0,0.0,var1


In [72]:
robjects.r('''
png("myisotropic_variogram.png",height=600,width=900)
print(plot(myiso))
dev.off()
''')

array([1], dtype=int32)

In [75]:
asd = robjects.r['myiso']

In [79]:
type(asd)

rpy2.robjects.vectors.DataFrame

In [80]:
print(asd)

        np      dist     gamma dir.hor dir.ver   id
1    97340  1.361011  21.90913       0       0 var1
2   342089  2.980286  59.87136       0       0 var1
3   599568  4.979613 105.45909       0       0 var1
4   781415  6.968800 147.17397       0       0 var1
5  1041462  8.972910 183.16559       0       0 var1
6  1148166 10.929269 216.56641       0       0 var1
7  1477391 12.936641 251.99665       0       0 var1
8  1567984 14.958394 285.56133       0       0 var1
9  1800696 16.961547 318.28864       0       0 var1
10 1956390 18.977989 351.51785       0       0 var1
11 2116441 20.988541 384.22676       0       0 var1
12 2178176 22.949285 416.17220       0       0 var1
13 2454400 24.941322 452.08826       0       0 var1
14 2557286 26.963809 487.00778       0       0 var1
15 2664181 28.968758 519.92306       0       0 var1
16 2841083 30.979062 549.14397       0       0 var1
17 2866723 32.978540 575.00261       0       0 var1
18 2984151 34.947485 598.00677       0       0 var1
19 3118331 3

In [82]:
asdf = robjects.r('myisomap <- variogram(R~1,mydata,width=2,cutoff=50,map=TRUE)')

In [83]:
import matplotlib.pyplot as plt
%matplotlib inline

In [90]:
robjects.r('''
png("myvariogram_map.png",height=600,width=600)
print(plot(myisomap))
dev.off()
''')

array([1], dtype=int32)

In [94]:
rainfall.head()

,R,x,y
0,3.35,3649.014148,1745.990418
1,5.65,3650.003652,1745.945446
2,5.51,3651.030377,1746.036880
3,6.24,3652.020031,1745.992300
4,7.64,3653.009769,1745.947910


In [131]:
asdff = pd.DataFrame(rainfall.sort_values(ascending=False, by='R'))
asdff = asdff[0:1499]
robjects.r.assign('sorted_rainfall', asdff)

R,x,y
142.420000,3683.988596,1688.034550
142.360000,3685.017002,1687.040719
139.960000,3700.991503,1686.991662


In [114]:
robjects.r('head(sorted_rainfall)')

,R,x,y
8155,142.42,3683.988596,1688.034550
8296,142.36,3685.017002,1687.040719
8312,139.96,3700.991503,1686.991662
8452,138.23,3701.021013,1686.037977
8453,137.54,3702.023914,1686.000409
8593,136.37,3701.971116,1685.009985


In [122]:
robjects.r('''
S <- sort(RAD24,index.return=TRUE,decreasing=TRUE)
''')

R object with classes: ('list',) mapped to:
<ListVector - Python:0x7f3d1b95c508 / R:0x565553fc5f48>
[FloatVector, IntVector]
  x: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f3d1b95ca48 / R:0x56555334dfe0>
[142.420000, 142.360000, 139.960000, 138.230000, ..., 0.000000, 0.000000, 0.000000, 0.000000]
  ix: <class 'rpy2.robjects.vectors.IntVector'>
  R object with classes: ('integer',) mapped to:
<IntVector - Python:0x7f3d1b95ce08 / R:0x565553327b00>
[8,156, 8,297, 8,313, 8,453, ..., 14,692, 15,530, 15,531, 15,671]

In [142]:
robjects.r('I')

array([ 8156,  8297,  8313, ..., 18432, 11133, 17031], dtype=int32)

In [124]:
robjects.r('head(RAD24)')

array([ 3.35,  5.65,  5.51,  6.24,  7.64,  6.69])

In [132]:
robjects.r.assign('myrad24', np.array(asdff.R))

142.420000,142.360000,139.960000,138.230000,...,60.680000,60.670000,60.660000,60.650000


In [138]:
robjects.r('''
S <- sort(RAD24,index.return=TRUE,decreasing=TRUE)
I <- S$ix[1:1499]
hat.anis <- estimateAnisotropy(data[I,],"R")		# anisotropy estimates (direction and ratio)
anis <- c(90-hat.anis$direction,1/hat.anis$ratio)
''')

array([ 99.95886125,   0.78840942])

In [139]:
robjects.r('''
directional_variograms <- variogram(R~1,data[wet,],width=2,cutoff=100,alpha=c(99.9,189.9),tol.hor=5)
png("directional_variograms.png",height=600,width=600)
plot(directional_variograms)
dev.off()
''')

array([1], dtype=int32)

In [140]:
robjects.r('''
initial_vario_sph <- vgm(psill=500,model="Sph",range=40,nugget=0)
fitted_vario_sph <- fit.variogram(isotropic_variogram,initial_vario_sph)
range  <- fitted_vario_sph$range[2]				# fitted range
nugget <- fitted_vario_sph$psill[1]				# fitted nugget
sill   <- sum(fitted_vario_sph$psill)			# fitted sill (total sill)
SSErr_sph <- attributes(fitted_vario_sph)$SSErr	# sum of squared errors (goodness of fit)

png("fitted_isotropic_variogram_sph.png",height=600,width=900)
print(plot(isotropic_variogram,fitted_vario_sph))
dev.off()
''')

array([1], dtype=int32)

In [141]:
robjects.r('''
initial_vario_exp <- vgm(psill=500,model="Exp",range=40/3,nugget=0)		# pseudo range = range at which you reach 95\% of the sill.
fitted_vario_exp <- fit.variogram(isotropic_variogram,initial_vario_exp)
SSErr_exp  <- attributes(fitted_vario_exp)$SSErr

# error is 2.4 times larger than for spherical model
# exponential is clearly not a good choice here.

png("fitted_isotropic_variogram_exp.png",height=600,width=900)
print(plot(isotropic_variogram,fitted_vario_exp))
dev.off()
''')

array([1], dtype=int32)